In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.2.1'

In [3]:
from sourmash.lca import lca_utils, LineagePair, taxlist

In [4]:
from collections import namedtuple
ComparisonInfo = namedtuple('ComparisonInfo',
                          'anchor_acc, lowest_common_rank, lowest_common_taxon, compare_accs')

In [5]:
metadata_file = "../gtdb-r202.metadata.csv.gz"

In [6]:
metadataDF = pd.read_csv(metadata_file, header=0, low_memory=False)
metadataDF["accession"] = metadataDF["accession"].str.replace("GB_", "").str.replace("RS_", "")
metadataDF

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_align_len,ssu_silva_blast_bitscore,ssu_silva_blast_evalue,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count
0,GCA_000200715.1,2,99.03,0.00,145,k__Archaea (UID2),103,0.00,1814397,88.719839,...,1470,2660,0,99.32,AF083072.21771.23243,Archaea;Crenarchaeota;Nitrososphaeria;Nitrosop...,229,18,40,0
1,GCA_000247545.1,0,100.00,0.00,174,k__Archaea (UID146),136,0.00,2178950,88.830863,...,1166,2154,0,100,CP003316.1430990.1433182,Archaea;Crenarchaeota;Thermoprotei;Thermoprote...,0,19,42,0
2,GCA_000306725.1,0,99.84,0.00,228,p__Euryarchaeota (UID49),153,0.00,2638662,85.872449,...,1472,2719,0,100,CP003083.1452985.1454458,Archaea;Halobacterota;Methanosarcinia;Methanos...,0,20,52,0
3,GCA_000375685.1,51,73.65,2.21,174,k__Archaea (UID146),136,66.67,1045795,86.867842,...,none,none,none,none,none,none,0,18,34,0
4,GCA_000387965.1,1,67.52,0.93,149,k__Archaea (UID2),107,100.00,548400,92.356039,...,1485,2743,0,100,APJZ01000002.58109.59609,Archaea;Nanoarchaeota;Nanoarchaeia;Nanoarchaea...,0,19,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258401,GCF_903986115.1,0,99.45,0.00,457,o__Actinomycetales (UID1812),227,0.00,2157429,89.805240,...,1516,2800,0,100,AJVH01000031.245.1773,Bacteria;Actinobacteriota;Actinobacteria;Coryn...,0,20,52,0
258402,GCF_903986355.1,0,99.45,0.00,457,o__Actinomycetales (UID1812),227,0.00,2155132,89.771507,...,1516,2800,0,100,AJVH01000031.245.1773,Bacteria;Actinobacteriota;Actinobacteria;Coryn...,0,20,52,0
258403,GCF_903986475.1,0,99.67,0.22,457,o__Actinomycetales (UID1812),227,100.00,2124579,89.280029,...,1006,1853,0,99.901,CP018331.1560127.1561658,Bacteria;Actinobacteriota;Actinobacteria;Coryn...,0,20,52,0
258404,GCF_903986495.1,0,99.45,0.13,457,o__Actinomycetales (UID1812),227,0.00,2202834,89.786075,...,842,1555,0,100,AJVH01000031.245.1773,Bacteria;Actinobacteriota;Actinobacteria;Coryn...,0,20,52,0


In [7]:
metadataDF["gtdb_taxonomy"]

0         d__Archaea;p__Thermoproteota;c__Nitrososphaeri...
1         d__Archaea;p__Thermoproteota;c__Thermoproteia;...
2         d__Archaea;p__Halobacteriota;c__Methanosarcini...
3         d__Archaea;p__Thermoproteota;c__Thermoproteia;...
4         d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...
                                ...                        
258401    d__Bacteria;p__Actinobacteriota;c__Actinomycet...
258402    d__Bacteria;p__Actinobacteriota;c__Actinomycet...
258403    d__Bacteria;p__Actinobacteriota;c__Actinomycet...
258404    d__Bacteria;p__Actinobacteriota;c__Actinomycet...
258405    d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
Name: gtdb_taxonomy, Length: 258406, dtype: object

In [8]:
gtdb_mDF = metadataDF[["accession", "gtdb_taxonomy"]].copy()
gtdb_mDF[["superkingdom","phylum","class","order","family","genus","species"]] = gtdb_mDF["gtdb_taxonomy"].str.split(pat=";", expand=True)
gtdb_mDF["signame"] = gtdb_mDF["accession"] + " " + gtdb_mDF["species"].str.replace("s__", "")
gtdb_mDF


,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
0,GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum,GCA_000200715.1 Cenarchaeum symbiosum
1,GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense,GCA_000247545.1 Pyrobaculum oguniense
2,GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanolobus,s__Methanolobus psychrophilus,GCA_000306725.1 Methanolobus psychrophilus
3,GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Gearchaeales,f__Gearchaeaceae,g__AAA261-N23,s__AAA261-N23 sp000375685,GCA_000375685.1 AAA261-N23 sp000375685
4,GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanopusillaceae,g__Nanopusillus,s__Nanopusillus stetteri,GCA_000387965.1 Nanopusillus stetteri
...,...,...,...,...,...,...,...,...,...,...
258401,GCF_903986115.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986115.1 Corynebacterium diphtheriae
258402,GCF_903986355.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986355.1 Corynebacterium diphtheriae
258403,GCF_903986475.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986475.1 Corynebacterium diphtheriae
258404,GCF_903986495.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986495.1 Corynebacterium diphtheriae


In [9]:
ncbi_mDF = metadataDF[["accession", "ncbi_taxonomy"]].copy()
ncbi_mDF[["superkingdom","phylum","class","order","family","genus","species"]] = ncbi_mDF["ncbi_taxonomy"].str.split(pat=";", expand=True)
ncbi_mDF["signame"] = ncbi_mDF["accession"] + " " + ncbi_mDF["species"].str.replace("s__", "")
ncbi_mDF


,accession,ncbi_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
0,GCA_000200715.1,d__Archaea;p__Thaumarchaeota;c__;o__Cenarchaea...,d__Archaea,p__Thaumarchaeota,c__,o__Cenarchaeales,f__Cenarchaeaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum,GCA_000200715.1 Cenarchaeum symbiosum
1,GCA_000247545.1,d__Archaea;p__Crenarchaeota;c__Thermoprotei;o_...,d__Archaea,p__Crenarchaeota,c__Thermoprotei,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense,GCA_000247545.1 Pyrobaculum oguniense
2,GCA_000306725.1,d__Archaea;p__Euryarchaeota;c__Methanomicrobia...,d__Archaea,p__Euryarchaeota,c__Methanomicrobia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanolobus,s__Methanolobus psychrophilus,GCA_000306725.1 Methanolobus psychrophilus
3,GCA_000375685.1,d__Archaea;p__Crenarchaeota;c__;o__;f__;g__;s__,d__Archaea,p__Crenarchaeota,c__,o__,f__,g__,s__,GCA_000375685.1
4,GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea,p__Nanoarchaeota,c__,o__Nanoarchaeales,f__Nanopusillaceae,g__Candidatus Nanobsidianus,s__Candidatus Nanobsidianus stetteri,GCA_000387965.1 Candidatus Nanobsidianus stetteri
...,...,...,...,...,...,...,...,...,...,...
258401,GCF_903986115.1,d__Bacteria;p__Actinobacteria;c__Actinobacteri...,d__Bacteria,p__Actinobacteria,c__Actinobacteria,o__Corynebacteriales,f__Corynebacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986115.1 Corynebacterium diphtheriae
258402,GCF_903986355.1,d__Bacteria;p__Actinobacteria;c__Actinobacteri...,d__Bacteria,p__Actinobacteria,c__Actinobacteria,o__Corynebacteriales,f__Corynebacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986355.1 Corynebacterium diphtheriae
258403,GCF_903986475.1,d__Bacteria;p__Actinobacteria;c__Actinobacteri...,d__Bacteria,p__Actinobacteria,c__Actinobacteria,o__Corynebacteriales,f__Corynebacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986475.1 Corynebacterium diphtheriae
258404,GCF_903986495.1,d__Bacteria;p__Actinobacteria;c__Actinobacteri...,d__Bacteria,p__Actinobacteria,c__Actinobacteria,o__Corynebacteriales,f__Corynebacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986495.1 Corynebacterium diphtheriae


In [10]:
# no class for the first accession!?
list(ncbi_mDF[ncbi_mDF["accession"] == "GCA_000200715.1" ]["ncbi_taxonomy"])

['d__Archaea;p__Thaumarchaeota;c__;o__Cenarchaeales;f__Cenarchaeaceae;g__Cenarchaeum;s__Cenarchaeum symbiosum']

In [11]:
# okay ncbi taxonomy can be weird. Let's ignore for now and just get reps for gtdb

In [12]:
species_representatives = metadataDF[metadataDF["gtdb_representative"] == "t"]["accession"]
species_representatives

0         GCA_000200715.1
1         GCA_000247545.1
2         GCA_000306725.1
3         GCA_000375685.1
4         GCA_000387965.1
               ...       
258347    GCF_902812405.1
258348    GCF_902812435.1
258351    GCF_902830275.1
258384    GCF_903643055.1
258385    GCF_903645355.1
Name: accession, Length: 47894, dtype: object

In [13]:
 gtdb_mDF.groupby("genus").first()

,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,species,signame
genus,,,,,,,,,
g__0-14-0-10-38-17,GCA_002774085.1,d__Bacteria;p__Patescibacteria;c__Microgenomat...,d__Bacteria,p__Patescibacteria,c__Microgenomatia,o__0-14-0-10-38-17,f__0-14-0-10-38-17,s__0-14-0-10-38-17 sp002774085,GCA_002774085.1 0-14-0-10-38-17 sp002774085
g__0-14-0-20-30-16,GCA_002779075.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__0-14-0-20-30-16,f__0-14-0-20-30-16,s__0-14-0-20-30-16 sp002779075,GCA_002779075.1 0-14-0-20-30-16 sp002779075
g__0-14-0-20-34-12,GCA_002779065.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__Iainarchaeales,f__0-14-0-20-34-12,s__0-14-0-20-34-12 sp002779065,GCA_002779065.1 0-14-0-20-34-12 sp002779065
g__0-14-0-20-40-13,GCA_002774285.1,d__Bacteria;p__Patescibacteria;c__WWE3;o__0-14...,d__Bacteria,p__Patescibacteria,c__WWE3,o__0-14-0-20-40-13,f__0-14-0-20-40-13,s__0-14-0-20-40-13 sp002774285,GCA_002774285.1 0-14-0-20-40-13 sp002774285
g__0-14-0-20-44-26,GCA_002779055.1,d__Bacteria;p__CG03;c__CG03;o__CG03;f__CG03;g_...,d__Bacteria,p__CG03,c__CG03,o__CG03,f__CG03,s__0-14-0-20-44-26 sp002779055,GCA_002779055.1 0-14-0-20-44-26 sp002779055
...,...,...,...,...,...,...,...,...,...
g__Zooshikella,GCF_000428585.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Zooshikellaceae,s__Zooshikella ganghwensis,GCF_000428585.1 Zooshikella ganghwensis
g__Zrk29,GCA_013745115.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Izemop...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Izemoplasmatales,f__UBA5603,s__Zrk29 sp013745115,GCA_013745115.1 Zrk29 sp013745115
g__Zunongwangia,GCA_003454375.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Flavobacteriales,f__Flavobacteriaceae,s__Zunongwangia profunda,GCA_003454375.1 Zunongwangia profunda


In [14]:
 gtdb_mDF.groupby("genus").nth(0)

,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,species,signame
genus,,,,,,,,,
g__0-14-0-10-38-17,GCA_002774085.1,d__Bacteria;p__Patescibacteria;c__Microgenomat...,d__Bacteria,p__Patescibacteria,c__Microgenomatia,o__0-14-0-10-38-17,f__0-14-0-10-38-17,s__0-14-0-10-38-17 sp002774085,GCA_002774085.1 0-14-0-10-38-17 sp002774085
g__0-14-0-20-30-16,GCA_002779075.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__0-14-0-20-30-16,f__0-14-0-20-30-16,s__0-14-0-20-30-16 sp002779075,GCA_002779075.1 0-14-0-20-30-16 sp002779075
g__0-14-0-20-34-12,GCA_002779065.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__Iainarchaeales,f__0-14-0-20-34-12,s__0-14-0-20-34-12 sp002779065,GCA_002779065.1 0-14-0-20-34-12 sp002779065
g__0-14-0-20-40-13,GCA_002774285.1,d__Bacteria;p__Patescibacteria;c__WWE3;o__0-14...,d__Bacteria,p__Patescibacteria,c__WWE3,o__0-14-0-20-40-13,f__0-14-0-20-40-13,s__0-14-0-20-40-13 sp002774285,GCA_002774285.1 0-14-0-20-40-13 sp002774285
g__0-14-0-20-44-26,GCA_002779055.1,d__Bacteria;p__CG03;c__CG03;o__CG03;f__CG03;g_...,d__Bacteria,p__CG03,c__CG03,o__CG03,f__CG03,s__0-14-0-20-44-26 sp002779055,GCA_002779055.1 0-14-0-20-44-26 sp002779055
...,...,...,...,...,...,...,...,...,...
g__Zooshikella,GCF_000428585.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Zooshikellaceae,s__Zooshikella ganghwensis,GCF_000428585.1 Zooshikella ganghwensis
g__Zrk29,GCA_013745115.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Izemop...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Izemoplasmatales,f__UBA5603,s__Zrk29 sp013745115,GCA_013745115.1 Zrk29 sp013745115
g__Zunongwangia,GCA_003454375.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Flavobacteriales,f__Flavobacteriaceae,s__Zunongwangia profunda,GCA_003454375.1 Zunongwangia profunda


In [15]:
# some of these only have a single member -- select nth(1) instead!
# (can only pairwise compare things with >1 genome)
gtdb_mDF[gtdb_mDF["genus"] == "g__0-14-0-10-38-17"]

,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
84644,GCA_002774085.1,d__Bacteria;p__Patescibacteria;c__Microgenomat...,d__Bacteria,p__Patescibacteria,c__Microgenomatia,o__0-14-0-10-38-17,f__0-14-0-10-38-17,g__0-14-0-10-38-17,s__0-14-0-10-38-17 sp002774085,GCA_002774085.1 0-14-0-10-38-17 sp002774085


In [16]:
gtdb_mDF[gtdb_mDF["genus"] == "g__0-14-0-20-30-16"]

,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
1698,GCA_002779075.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__0-14-0-20-30-16,f__0-14-0-20-30-16,g__0-14-0-20-30-16,s__0-14-0-20-30-16 sp002779075,GCA_002779075.1 0-14-0-20-30-16 sp002779075
2337,GCA_903916665.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__0-14-0-20-30-16,f__0-14-0-20-30-16,g__0-14-0-20-30-16,s__0-14-0-20-30-16 sp903916665,GCA_903916665.1 0-14-0-20-30-16 sp903916665


In [17]:
 gtdb_mDF.groupby("genus").nth(1)

,accession,gtdb_taxonomy,superkingdom,phylum,class,order,family,species,signame
genus,,,,,,,,,
g__0-14-0-20-30-16,GCA_903916665.1,d__Archaea;p__Iainarchaeota;c__Iainarchaeia;o_...,d__Archaea,p__Iainarchaeota,c__Iainarchaeia,o__0-14-0-20-30-16,f__0-14-0-20-30-16,s__0-14-0-20-30-16 sp903916665,GCA_903916665.1 0-14-0-20-30-16 sp903916665
g__0-14-0-20-59-11,GCA_002763345.1,d__Archaea;p__Micrarchaeota;c__Micrarchaeia;o_...,d__Archaea,p__Micrarchaeota,c__Micrarchaeia,o__UBA8480,f__UBA93,s__0-14-0-20-59-11 sp002763345,GCA_002763345.1 0-14-0-20-59-11 sp002763345
g__0-14-0-80-42-15,GCA_002780005.1,d__Bacteria;p__Omnitrophota;c__Koll11;o__UBA15...,d__Bacteria,p__Omnitrophota,c__Koll11,o__UBA1560_A,f__0-14-0-80-42-15,s__0-14-0-80-42-15 sp002780005,GCA_002780005.1 0-14-0-80-42-15 sp002780005
g__0-14-3-00-41-53,GCA_002785245.1,d__Bacteria;p__Nitrospirota;c__Thermodesulfovi...,d__Bacteria,p__Nitrospirota,c__Thermodesulfovibrionia,o__Thermodesulfovibrionales,f__SM23-35,s__0-14-3-00-41-53 sp002780895,GCA_002785245.1 0-14-3-00-41-53 sp002780895
g__01-FULL-36-15b,GCA_001782035.1,d__Bacteria;p__Patescibacteria;c__Microgenomat...,d__Bacteria,p__Patescibacteria,c__Microgenomatia,o__Levybacterales,f__UBA12049,s__01-FULL-36-15b sp001782035,GCA_001782035.1 01-FULL-36-15b sp001782035
...,...,...,...,...,...,...,...,...,...
g__Zoogloea,GCA_009026175.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Rhodocyclaceae,s__Zoogloea sp009026175,GCA_009026175.1 Zoogloea sp009026175
g__Zooshikella,GCF_002261065.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Zooshikellaceae,s__Zooshikella ganghwensis,GCF_002261065.1 Zooshikella ganghwensis
g__Zunongwangia,GCA_002690095.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Flavobacteriales,f__Flavobacteriaceae,s__Zunongwangia profunda,GCA_002690095.1 Zunongwangia profunda


In [18]:
genus_anchors = gtdb_mDF.groupby("genus").nth(1)["accession"]
genus_anchors

genus
g__0-14-0-20-30-16    GCA_903916665.1
g__0-14-0-20-59-11    GCA_002763345.1
g__0-14-0-80-42-15    GCA_002780005.1
g__0-14-3-00-41-53    GCA_002785245.1
g__01-FULL-36-15b     GCA_001782035.1
                           ...       
g__Zoogloea           GCA_009026175.1
g__Zooshikella        GCF_002261065.1
g__Zunongwangia       GCA_002690095.1
g__Zymobacter         GCF_003610015.1
g__Zymomonas          GCF_006716005.1
Name: accession, Length: 7435, dtype: object

In [19]:
comparison_ranks = lca_utils.taxlist(include_strain=False)
print(list(comparison_ranks))

['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']


In [20]:
rank="genus"
gtdb_mDF.groupby(rank).nth(1)["accession"]

genus
g__0-14-0-20-30-16    GCA_903916665.1
g__0-14-0-20-59-11    GCA_002763345.1
g__0-14-0-80-42-15    GCA_002780005.1
g__0-14-3-00-41-53    GCA_002785245.1
g__01-FULL-36-15b     GCA_001782035.1
                           ...       
g__Zoogloea           GCA_009026175.1
g__Zooshikella        GCF_002261065.1
g__Zunongwangia       GCA_002690095.1
g__Zymobacter         GCF_003610015.1
g__Zymomonas          GCF_006716005.1
Name: accession, Length: 7435, dtype: object

In [21]:
metadataDF[metadataDF["gtdb_representative"] == "t"][["accession", "gtdb_taxonomy"]]

,accession,gtdb_taxonomy
0,GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...
1,GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...
2,GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...
3,GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...
4,GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...
...,...,...
258347,GCF_902812405.1,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...
258348,GCF_902812435.1,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...
258351,GCF_902830275.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
258384,GCF_903643055.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...


In [22]:
rank = "superkingdom"
reps = gtdb_mDF.groupby(rank).nth(1)[["accession"]]#, rank]]
reps

,accession
superkingdom,
d__Archaea,GCA_000247545.1
d__Bacteria,GCA_000016465.1


In [23]:
gtdb_mDF.set_index("accession", inplace=True)
gtdb_mDF

,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
accession,,,,,,,,,
GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum,GCA_000200715.1 Cenarchaeum symbiosum
GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense,GCA_000247545.1 Pyrobaculum oguniense
GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanolobus,s__Methanolobus psychrophilus,GCA_000306725.1 Methanolobus psychrophilus
GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Gearchaeales,f__Gearchaeaceae,g__AAA261-N23,s__AAA261-N23 sp000375685,GCA_000375685.1 AAA261-N23 sp000375685
GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanopusillaceae,g__Nanopusillus,s__Nanopusillus stetteri,GCA_000387965.1 Nanopusillus stetteri
...,...,...,...,...,...,...,...,...,...
GCF_903986115.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986115.1 Corynebacterium diphtheriae
GCF_903986355.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986355.1 Corynebacterium diphtheriae
GCF_903986475.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Mycobacteriales,f__Mycobacteriaceae,g__Corynebacterium,s__Corynebacterium diphtheriae,GCF_903986475.1 Corynebacterium diphtheriae


## select representatives randomly?

In [ ]:
random_representatives_at_ranks={}
for rank in lca_utils.taxlist(include_strain=False):
    print(rank)
    if rank == "species":
        reps = metadataDF[metadataDF["gtdb_representative"] == "t"]["accession"]
    else:
        # this selects randomly. Probably want to use the representatives instead!?
        reps = gtdb_mDF.groupby(rank, as_index=False).nth(1).index 
    random_representatives_at_ranks[rank] = reps

In [ ]:
# test getting comparison acc on superk = []
for acc in random_representatives_at_ranks["superkingdom"]:
    lowest_common_taxon = gtdb_mDF.at[acc, "superkingdom"]
    print(lowest_common_taxon)
    acc_to_compare = list(gtdb_mDF[gtdb_mDF["superkingdom"] == lowest_common_taxon].index)
    print(len(acc_to_compare))
    # remove anchor accession
    acc_to_compare.remove(acc)
    print(len(acc_to_compare))
    acc_to_compare = ";".join(acc_to_compare)
    print(acc_to_compare[:100])
    comparisons.append(ComparisonInfo(acc, "superkingdom", lowest_common_taxon, acc_to_compare))

In [ ]:
# actually get all comparisons
rand_comparisons = []
for rank in lca_utils.taxlist(include_strain=False):
    for acc in representatives_at_ranks[rank]:
        lowest_common_taxon = gtdb_mDF.at[acc, rank]
        acc_to_compare = list(gtdb_mDF[gtdb_mDF[rank] == lowest_common_taxon].index)
        # remove anchor accession
        acc_to_compare.remove(acc)
        acc_to_compare = ";".join(acc_to_compare)
        rand_comparisons.append(ComparisonInfo(acc, rank, lowest_common_taxon, acc_to_compare))

In [ ]:
rand_compareDF = pd.DataFrame.from_records(rand_comparisons, columns = ComparisonInfo._fields)
rand_compareDF.sort_values("lowest_common_rank", inplace=True)

## Alteratively, ensure that we're always comparing to one of the representative genomes?

In [25]:
representatives_only = metadataDF[metadataDF["gtdb_representative"] == "t"]
representatives_only

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_align_len,ssu_silva_blast_bitscore,ssu_silva_blast_evalue,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count
0,GCA_000200715.1,2,99.03,0.00,145,k__Archaea (UID2),103,0.00,1814397,88.719839,...,1470,2660,0,99.32,AF083072.21771.23243,Archaea;Crenarchaeota;Nitrososphaeria;Nitrosop...,229,18,40,0
1,GCA_000247545.1,0,100.00,0.00,174,k__Archaea (UID146),136,0.00,2178950,88.830863,...,1166,2154,0,100,CP003316.1430990.1433182,Archaea;Crenarchaeota;Thermoprotei;Thermoprote...,0,19,42,0
2,GCA_000306725.1,0,99.84,0.00,228,p__Euryarchaeota (UID49),153,0.00,2638662,85.872449,...,1472,2719,0,100,CP003083.1452985.1454458,Archaea;Halobacterota;Methanosarcinia;Methanos...,0,20,52,0
3,GCA_000375685.1,51,73.65,2.21,174,k__Archaea (UID146),136,66.67,1045795,86.867842,...,none,none,none,none,none,none,0,18,34,0
4,GCA_000387965.1,1,67.52,0.93,149,k__Archaea (UID2),107,100.00,548400,92.356039,...,1485,2743,0,100,APJZ01000002.58109.59609,Archaea;Nanoarchaeota;Nanoarchaeia;Nanoarchaea...,0,19,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258347,GCF_902812405.1,0,96.30,0.00,161,k__Bacteria (UID1452),108,0.00,2102236,90.606513,...,1481,2329,0,95.138,JN023677.1.1476,Bacteria;WPS-2;uncultured bacterium,0,20,44,1
258348,GCF_902812435.1,0,91.47,1.27,295,p__Firmicutes (UID1022),158,0.00,1692473,86.869710,...,none,none,none,none,none,none,0,19,38,1
258351,GCF_902830275.1,0,100.00,1.41,769,g__Burkholderia (UID4006),248,0.00,7022950,87.432990,...,1529,2819,0,99.935,CP013460.203788.205323,Bacteria;Proteobacteria;Gammaproteobacteria;Bu...,0,20,60,0
258384,GCF_903643055.1,0,83.83,5.46,427,o__Burkholderiales (UID4000),214,15.79,3893605,88.147867,...,none,none,none,none,none,none,0,16,28,0


In [26]:
gtdb_mDF_reps_only = gtdb_mDF[(gtdb_mDF.index).isin(representatives_only["accession"])]
gtdb_mDF_reps_only

,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
accession,,,,,,,,,
GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum,GCA_000200715.1 Cenarchaeum symbiosum
GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense,GCA_000247545.1 Pyrobaculum oguniense
GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanolobus,s__Methanolobus psychrophilus,GCA_000306725.1 Methanolobus psychrophilus
GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Gearchaeales,f__Gearchaeaceae,g__AAA261-N23,s__AAA261-N23 sp000375685,GCA_000375685.1 AAA261-N23 sp000375685
GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanopusillaceae,g__Nanopusillus,s__Nanopusillus stetteri,GCA_000387965.1 Nanopusillus stetteri
...,...,...,...,...,...,...,...,...,...
GCF_902812405.1,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria,p__Eremiobacterota,c__Eremiobacteria,o__Eremiobacterales,f__Eremiobacteraceae,g__CADDZI01,s__CADDZI01 sp902812405,GCF_902812405.1 CADDZI01 sp902812405
GCF_902812435.1,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria,p__Firmicutes_B,c__Syntrophomonadia,o__Thermacetogeniales,f__Thermacetogeniaceae,g__DULK01,s__DULK01 sp902812435,GCF_902812435.1 DULK01 sp902812435
GCF_902830275.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Burkholderia,s__Burkholderia stagnalis,GCF_902830275.1 Burkholderia stagnalis


In [156]:
# select representatives from representative genomes only (use **gtdb_mDF_reps_only** DF)
# NOTE -- assumes that every species has a representative. I think this is a valid assumption...
representatives_at_ranks={}
for rank in lca_utils.taxlist(include_strain=False):
    print(rank)
    if rank == "species":
        reps = gtdb_mDF_reps_only.groupby(rank, as_index=False).nth(0).index
        print(len(reps))
    else:
        reps = gtdb_mDF_reps_only.groupby(rank, as_index=False).nth(0).index
        print(len(reps))
    representatives_at_ranks[rank] = reps

superkingdom
2
phylum
169
class
419
order
1312
family
3264
genus
12888
species
47894


In [157]:
# test get comparison accessions on superk
rep_comparisons = []
for acc in representatives_at_ranks["superkingdom"]:
    lowest_common_taxon = gtdb_mDF.at[acc, "superkingdom"]
    print(lowest_common_taxon)
    acc_to_compare = list(gtdb_mDF[gtdb_mDF["superkingdom"] == lowest_common_taxon].index)
    print(len(acc_to_compare))
    # remove anchor accession
    acc_to_compare.remove(acc)
    print(len(acc_to_compare))
    acc_to_compare = ";".join(acc_to_compare)
    print(acc_to_compare[:100])
    rep_comparisons.append(ComparisonInfo(acc, "superkingdom", lowest_common_taxon, acc_to_compare))

d__Archaea
4316
4315
GCA_000247545.1;GCA_000306725.1;GCA_000375685.1;GCA_000387965.1;GCA_000402775.1;GCA_000402965.1;GCA_
d__Bacteria
254090
254089
GCA_000013845.2;GCA_000016465.1;GCA_000152525.1;GCA_000172955.1;GCA_000188315.1;GCA_000196855.1;GCA_


In [158]:
# get accs to compare to:
rep_comparisons = []
for rank in lca_utils.taxlist(include_strain=False):
    for acc in representatives_at_ranks[rank]:
        lowest_common_taxon = gtdb_mDF.at[acc, rank]
        acc_to_compare = list(gtdb_mDF[gtdb_mDF[rank] == lowest_common_taxon].index)
        # if there's only one genome in this group, nothing to compare!
        if len(acc_to_compare) == 1:
            continue
         # remove anchor accession
        acc_to_compare.remove(acc)
        acc_to_compare = ";".join(acc_to_compare)
        rep_comparisons.append(ComparisonInfo(acc, rank, lowest_common_taxon, acc_to_compare))

KeyboardInterrupt: 

In [ ]:
rep_compareDF = pd.DataFrame.from_records(rep_comparisons, columns = ComparisonInfo._fields)
rep_compareDF.sort_values("lowest_common_rank", inplace=True)

In [ ]:
rep_compareDF.sort_values("lowest_common_rank", inplace=True)

In [ ]:
compareDF.to_csv(args.output_csv, index=False)

In [159]:
for rank in lca_utils.taxlist(include_strain=False):
    print(rank)

superkingdom
phylum
class
order
family
genus
species


In [27]:
gtdb_mDF_reps_only = gtdb_mDF[(gtdb_mDF.index).isin(representatives_only["accession"])].copy()
gtdb_mDF_reps_only

,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame
accession,,,,,,,,,
GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum,GCA_000200715.1 Cenarchaeum symbiosum
GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense,GCA_000247545.1 Pyrobaculum oguniense
GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanolobus,s__Methanolobus psychrophilus,GCA_000306725.1 Methanolobus psychrophilus
GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Gearchaeales,f__Gearchaeaceae,g__AAA261-N23,s__AAA261-N23 sp000375685,GCA_000375685.1 AAA261-N23 sp000375685
GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanopusillaceae,g__Nanopusillus,s__Nanopusillus stetteri,GCA_000387965.1 Nanopusillus stetteri
...,...,...,...,...,...,...,...,...,...
GCF_902812405.1,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria,p__Eremiobacterota,c__Eremiobacteria,o__Eremiobacterales,f__Eremiobacteraceae,g__CADDZI01,s__CADDZI01 sp902812405,GCF_902812405.1 CADDZI01 sp902812405
GCF_902812435.1,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria,p__Firmicutes_B,c__Syntrophomonadia,o__Thermacetogeniales,f__Thermacetogeniaceae,g__DULK01,s__DULK01 sp902812435,GCF_902812435.1 DULK01 sp902812435
GCF_902830275.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Burkholderia,s__Burkholderia stagnalis,GCF_902830275.1 Burkholderia stagnalis


In [40]:
taxonomy="gtdb_taxonomy"
def taxonomy_at_ranks(row):
    rank_lineages = []
    lin = lca_utils.make_lineage(row[taxonomy])
    for rank in lca_utils.taxlist(include_strain=False):
        rank_lin = lca_utils.pop_to_rank(lin, rank)
        rank_lin = ";".join(lca_utils.zip_lineage(rank_lin, include_strain=False, truncate_empty=True))
        row[rank] = rank_lin
    return row

In [41]:
gtdb_mDF_reps_only = gtdb_mDF_reps_only.apply(taxonomy_at_ranks,axis=1)


In [42]:
gtdb_mDF_reps_only

,gtdb_taxonomy,superkingdom,phylum,class,order,family,genus,species,signame,rank
accession,,,,,,,,,,
GCA_000200715.1,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea,d__Archaea;p__Thermoproteota,d__Archaea;p__Thermoproteota;c__Nitrososphaeria,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,GCA_000200715.1 Cenarchaeum symbiosum,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...
GCA_000247545.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,d__Archaea;p__Thermoproteota,d__Archaea;p__Thermoproteota;c__Thermoproteia,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,GCA_000247545.1 Pyrobaculum oguniense,d__Archaea;p__Thermoproteota;c__Thermoproteia;...
GCA_000306725.1,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea,d__Archaea;p__Halobacteriota,d__Archaea;p__Halobacteriota;c__Methanosarcinia,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea;p__Halobacteriota;c__Methanosarcini...,d__Archaea;p__Halobacteriota;c__Methanosarcini...,GCA_000306725.1 Methanolobus psychrophilus,d__Archaea;p__Halobacteriota;c__Methanosarcini...
GCA_000375685.1,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea,d__Archaea;p__Thermoproteota,d__Archaea;p__Thermoproteota;c__Thermoproteia,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,GCA_000375685.1 AAA261-N23 sp000375685,d__Archaea;p__Thermoproteota;c__Thermoproteia;...
GCA_000387965.1,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea,d__Archaea;p__Nanoarchaeota,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,GCA_000387965.1 Nanopusillus stetteri,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...
...,...,...,...,...,...,...,...,...,...,...
GCF_902812405.1,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria,d__Bacteria;p__Eremiobacterota,d__Bacteria;p__Eremiobacterota;c__Eremiobacteria,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,GCF_902812405.1 CADDZI01 sp902812405,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...
GCF_902812435.1,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria,d__Bacteria;p__Firmicutes_B,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadia,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...,GCF_902812435.1 DULK01 sp902812435,d__Bacteria;p__Firmicutes_B;c__Syntrophomonadi...
GCF_902830275.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,d__Bacteria;p__Proteobacteria,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,GCF_902830275.1 Burkholderia stagnalis,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
